# Chapter 3: Cloud Computing .

The goal of this chapter is to delve into different libraries and concept used in cloud computing.

Structure:
- [SSH](#SSH)
- [APACHE](#APACHE)

<a name="SSH"></a>
## SSH

"Secure Shell (SSH) is a cryptographic network protocol for operating network services securely over an unsecured network." (Wikipedia). 
This protocol is composed of a client side called ssh and a server side called SSHD. The client side logs into a remote machine and can execute command on said machine. This logging can be done using ssh keys which provides a more secure method of authentification than using a password. Even though SSH was meant for unix systems at the beginning, it is popular enough that a tool for windows was created: OpenSSH. 2 years ago windows launched their terminal which also supports SSH. 

The goal of this section will only be to create the SSH encryption keys for the next section "Git". 

If you are interested in creating an sshd or connecting to one, here are some tutorials that go more in depth:
- Understanding the idea behind ssh:
    * https://www.hostinger.com/tutorials/ssh-tutorial-how-does-ssh-work
- SSH on windows:
    * https://winscp.net/eng/docs/guide_windows_openssh_server
    * https://github.com/PowerShell/Win32-OpenSSH/wiki/Install-Win32-OpenSSH
    * https://www.pugetsystems.com/labs/hpc/How-To-Use-SSH-Client-and-Server-on-Windows-10-1470/

Now our goal will be to create our own SSHkeys. To do this you have two options:
- PuTTy https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html. You can then use PuTTYgen to generate your keys (Some server will require a specific format, for example the OpenSSH format. to convert the one you got from PuTTY you can use this command line in the terminal) .

```console
ssh-keygen -i -f keyfile.pub > newkeyfile.pub
```

- Using your terminal

```console
ssh-keygen -t rsa -C "your_email"
```

Make sure this email is the same as on the websites you will use this SSH key on.

If you want an additional security make sure to create a passphrase. If you do and dealing with it is to annoying make sure to create an agent (on windows: https://stackoverflow.com/questions/52113738/starting-ssh-agent-on-windows-10-fails-unable-to-start-ssh-agent-service-erro and dont forget to have OpenSSH)

You will have a public and a private key. The private key can be use to generate the public key. **Never share your private key**. You need to give your public key to the server side but never the private. You will use the private when login onto the SSHD.

If you missed any step or have any trouble you can also use this very well done tutorial: https://upcloud.com/community/tutorials/use-ssh-keys-authentication/

Make sure to know where you save your private key for the following section.

### Github and SSH
Now let us work again with github. You can add your public SSH key in the settings. (note: github expect the OpenSSH format).
You can test your connection using

```console
ssh -T git@github.com
```


<a name="APACHE"></a>
## APACHE
